In [10]:
import ast
import re
import json
import tqdm
import pickle

from pathlib import Path
from datetime import datetime
from pydriller import RepositoryMining


GIT_REPO_PATH = 'data/django/'
DATA_PATH = 'data/'
SINCE_DATETIME = datetime(2020, 7, 7, 0, 0, 0)
TO_DATETIME = datetime(2020, 7, 8, 0, 0, 0)

FIX_MSG_PATTERN = r'fix|error|bug|issue|mistake|incorrect|fault|defect|flaw|type'

# total_django_comits = 28542
# fix_django_commits  = 18820

In [5]:
class CodeChange():
    def __init__(self, commit_hash, source_code_before, source_code_after, git_diff):
        self.commit_hash = commit_hash
        self.before = source_code_before
        self.after = source_code_after
        self.git_diff = git_diff

In [7]:
%%time
repo = RepositoryMining(GIT_REPO_PATH)
changes = []

for commit in tqdm.tqdm(repo.traverse_commits()):
    if re.search(FIX_MSG_PATTERN, commit.msg, re.IGNORECASE):
        modifications = commit.modifications
        for mod in modifications:
            if (mod.filename.endswith('.py')):
                code_change = CodeChange(commit.hash, 
                                         mod.source_code_before, 
                                         mod.source_code, 
                                         mod.diff_parsed)
                changes.append(code_change)

28542it [14:25, 32.97it/s]


Wall time: 14min 26s


In [8]:
len(changes)

41185

In [11]:
with open('data/django-fixes.pickle', 'wb') as handle:
    pickle.dump(changes, handle, protocol=pickle.HIGHEST_PROTOCOL)